In [29]:
from dotenv import load_dotenv
import os
import openai
from openai import OpenAI
import pandas as pd
import json

In [2]:
load_dotenv()

API_KEY = os.getenv('API_KEY')

In [9]:
client = OpenAI(
    api_key=os.environ.get("API_KEY"),
)


### Print a list of openAI models

In [19]:
models = client.models.list()
print (models)

SyncPage[Model](data=[Model(id='dall-e-3', created=1698785189, object='model', owned_by='system'), Model(id='gpt-4-1106-preview', created=1698957206, object='model', owned_by='system'), Model(id='whisper-1', created=1677532384, object='model', owned_by='openai-internal'), Model(id='davinci-002', created=1692634301, object='model', owned_by='system'), Model(id='gpt-4o-2024-05-13', created=1715368132, object='model', owned_by='system'), Model(id='babbage-002', created=1692634615, object='model', owned_by='system'), Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'), Model(id='gpt-3.5-turbo-16k', created=1683758102, object='model', owned_by='openai-internal'), Model(id='tts-1-hd-1106', created=1699053533, object='model', owned_by='system'), Model(id='tts-1-hd', created=1699046015, object='model', owned_by='system'), Model(id='gpt-3.5-turbo-1106', created=1698959748, object='model', owned_by='system'), Model(id='gpt-3.5-turbo-instruct-0914', created=1694122472, obj

---

### Completion API

In [38]:
response = client.chat.completions.create(
    model = 'gpt-4-turbo', 
    messages = [{"role": "system", "content": 'Act as an AI Assistant'}, # system: tell the agent what role it needs to play
                {"role": "user", "content": 'Explain Machine Learning to a 5 year old'}],  
    max_tokens = 20,
    temperature = 0.9, # this is the degree of randomness of the model's output,
    n = 5 # indicate how many outputs needed
    )

print(response)

ChatCompletion(id='chatcmpl-9XrbMM8LipG2hwYkv0qUAr8dPGWic', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='Okay! Imagine you have a smart robot friend who really wants to help you clean up your room.', role='assistant', function_call=None, tool_calls=None)), Choice(finish_reason='length', index=1, logprobs=None, message=ChatCompletionMessage(content='Imagine you have a magic coloring book. Each time you color a picture, the book learns how you', role='assistant', function_call=None, tool_calls=None)), Choice(finish_reason='length', index=2, logprobs=None, message=ChatCompletionMessage(content='Sure! Imagine you have a smart robot friend who wants to learn to tell the difference between cats and', role='assistant', function_call=None, tool_calls=None)), Choice(finish_reason='length', index=3, logprobs=None, message=ChatCompletionMessage(content='Imagine you have a magic box that can learn to do things just by watching you do them a

---

### Tactics

In [35]:
def get_response_to_prompt (prompt):
    response = client.chat.completions.create(
        model = 'gpt-4-turbo', 
        messages = [{"role": "user", "content": prompt}],  
        temperature = 0.9 # this is the degree of randomness of the model's output
        )
    return response.choices[0].message.content

**Tactic 1**: Use delimiters and ask for structured outpuit

In [36]:
text = f"""
I had a fantastic time at Standford CSB! Learned a lot and also made great nwe friends!
---
I had a aweful time at Standford CSB! Cannot learned a lot or made great nwe friends!
---
I had a both a fantastic and aweful time at Standford CSB! Cannot learned a lot or made great nwe friends!

"""

prompt = f"""Classify each sentence in ```{text}```, delimited by three dashes (---), as having either a positive or negative or neutral sentiment. \
    Only output the number of each sentence along with the sentiment. No  need to output the sentence itself.

"""

In [37]:
response = get_response_to_prompt(prompt)
print(response)

1. Positive
2. Negative
3. Negative


**Tactic 2**: "Few-shot" prompting: give successful examples of completing tasks, then ask the model to perform the task

In [12]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \ 
valley flows from a modest spring; the \ 
grandest symphony originates from a single note; \ 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about Japan.
"""
response = get_response_to_prompt(prompt)
print(response)

<grandparent>: Japan is a land of rich traditions and ancient customs, where respect and honor are highly valued. The country is known for its beautiful cherry blossoms, delicious cuisine, and intricate artwork such as origami and bonsai. Japan is also a land of contrasts, with bustling cities like Tokyo and serene natural landscapes like Mount Fuji. It is a place where past and present seamlessly blend together, creating a truly unique and captivating culture.


**Tactic 3**: check whether conditions are satisfied. Check assumptions required to do the task

In [13]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \ 
water boiling. While that's happening, \ 
grab a cup and put a tea bag in it. Once the water is \ 
hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a \ 
few minutes, take out the tea bag. If you \ 
like, you can add some sugar or milk to taste. \ 
And that's it! You've got yourself a delicious \ 
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_response_to_prompt(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Once the water is hot enough, pour it over the tea bag.
Step 4 - Let it sit for a bit so the tea can steep.
Step 5 - After a few minutes, take out the tea bag.
Step 6 - Add some sugar or milk to taste.
Step 7 - Enjoy your delicious cup of tea.


In [14]:
text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \ 
walk in the park. The flowers are blooming, and the \ 
trees are swaying gently in the breeze. People \ 
are out and about, enjoying the lovely weather. \ 
Some are having picnics, while others are playing \ 
games or simply relaxing on the grass. It's a \ 
perfect day to spend time outdoors and appreciate the \ 
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
response = get_response_to_prompt(prompt)
print("Completion for Text 2:")
print(response)

Completion for Text 2:
No steps provided.


**Tactic 4**: instruct the model to work out its own solution, before rushing to a conclusion

In [15]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""

response = get_response_to_prompt(prompt)
print(response)

The student's solution is incorrect. The total cost should be calculated as follows:
1. Land cost: $100/square foot * x square feet = $100x
2. Solar panel cost: $250/square foot * x square feet = $250x
3. Maintenance cost: $100,000 + $10/square foot * x square feet = $100,000 + $10x
Total cost: $100x + $250x + $100,000 + $10x = $360x + $100,000

Therefore, the correct total cost for the first year of operations as a function of the number of square feet is $360x + $100,000.


Note that the student's solution is actually not correct. We can fix this by instructing the model to work out its own solution first.

In [16]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem including the final total. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_response_to_prompt(prompt)
print(response)

First, let's calculate the total cost for the first year of operations:
- Land cost = $100 / square foot
- Solar panel cost = $250 / square foot
- Maintenance cost = $100,000 + $10 / square foot

Total cost for 1st year of operations = Land cost + Solar panel cost + Maintenance cost
Total cost = $100x + $250x + $100,000 + $10x
Total cost = $360x + $100,000

Therefore, the actual solution for the total cost for the first year of operations as a function of the number of square feet is 360x + 100,000.

Is the student's solution the same as the actual solution just calculated:
```
No
```
Student grade:
```
Incorrect
```


**Tactic 5**: iteratively improve your prompt

In [17]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture, 
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100) 
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, 
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities: 
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests 

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

In [19]:
## Without limit
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_response_to_prompt(prompt)
print(response)

Introducing our sleek and stylish mid-century inspired office chair that is part of a beautiful family of furniture pieces perfect for any home or business setting. This chair comes in several shell color and base finish options, allowing you to customize it to match your existing decor seamlessly.

The chair is available in two upholstery options: plastic back and front upholstery or full upholstery in a variety of fabric and leather choices. You can also choose between a chair with or without armrests, and select between soft or hard-floor caster options.

Constructed with a durable 5-wheel plastic coated aluminum base and featuring a pneumatic chair adjustment for easy raise/lower action, this chair is both functional and ergonomic. The seat is made with high-quality HD36 foam and comes in two densities: medium or high. 

Made in Italy, this chair is not only stylish but also built to last. With its modern design and quality craftsmanship, this chair is the perfect addition to any w

In [21]:
## Specify the number of words/sentences/charaters, if response text is too long
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_response_to_prompt(prompt)
print(response)

Introducing our versatile and stylish office chair, available in a variety of colors and finishes to suit any space. With adjustable features and high-quality materials, this chair is perfect for both home and office use. Made in Italy, it's a statement piece for any setting.


In [22]:
## With limit
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Use at most3 sentences.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_response_to_prompt(prompt)
print(response)

Introducing our beautifully designed mid-century inspired office chair, crafted with a 5-wheel plastic coated aluminum base for easy mobility. Available in a variety of shell colors and base finishes, with options for plastic or full upholstery in a range of fabrics and leathers. Perfect for home or business settings, this chair also offers customizable features such as armrests, seat foam densities, and caster options. Made in Italy with high-quality materials, this chair combines style, comfort, and functionality for a truly versatile seating solution.


In [29]:
## Add Specify the intended audience to make the content more relelvant
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_response_to_prompt(prompt)
print(response, 20)

Introducing our sleek and versatile office chair, part of a stylish mid-century inspired furniture collection. Constructed with a durable aluminum base, a comfortable HD36 foam seat, and a stylish shell made of cast aluminum with nylon coating. Available in various colors and finishes to suit any space. 

Product IDs: SWC-100, SWC-110 20


In [33]:
## Ask it to extract information and organize it in a table.
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

After the description, include a table that gives the 
product's dimensions. The table should have two columns.
In the first column include the name of the dimension. 
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website. 
Place the description in a <div> element.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_response_to_prompt(prompt)
print(response)

<div>
    <p>This mid-century inspired office chair is the perfect addition to any workspace. With a variety of shell colors and base finishes to choose from, you can customize this chair to fit your unique style. The chair is available with plastic back and front upholstery or full upholstery in a selection of fabric and leather options. The 5-wheel plastic coated aluminum base provides stability while the pneumatic chair adjust allows for easy raise and lower action. Whether for home or business settings, this chair is sure to impress. Made in Italy with high-quality materials, this chair is both stylish and functional.</p>
    
    <p>Product IDs: SWC-100, SWC-110</p>
    
    <h2>Product Dimensions</h2>
    <table>
        <tr>
            <td>WIDTH</td>
            <td>20.87"</td>
        </tr>
        <tr>
            <td>DEPTH</td>
            <td>20.08"</td>
        </tr>
        <tr>
            <td>HEIGHT</td>
            <td>31.50"</td>
        </tr>
        <tr>
           

In [34]:
from IPython.display import display, HTML
display(HTML(response))

WIDTH,"20.87"""
DEPTH,"20.08"""
HEIGHT,"31.50"""
SEAT HEIGHT,"17.32"""
SEAT DEPTH,"16.14"""
